Pytorch code

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
transaction_rate = 0.005

def newWealth_withTC(W,r,pi_pre,pi_new):
    # W_new = torch.scalar_tensor(0,requires_grad=True)
    # #W_new = 0
    # #W = torch.scalar_tensor(W,requires_grad=True)
    # r = torch.tensor(r,requires_grad=True)
    # for i in range(n+1):
    #     W_new += W*pi_pre[i]*(1+r[i])
    # W_new = W_new - W_new*transaction_rate*torch.norm(pi_new-pi_pre,p = 1)
    # return torch.tensor(W_new,requires_grad=True)
    W = torch.tensor(W, requires_grad=True)
    r = torch.tensor(r, requires_grad=True)
    pi_pre = torch.tensor(pi_pre, requires_grad=True)
    pi_new = torch.tensor(pi_new, requires_grad=True)

    n = len(r) - 1
    W_new = torch.scalar_tensor(0.0, requires_grad=True)

    # Accumulate values in a new tensor without in-place operations
    for i in range(n + 1):
        W_new = W_new + W * pi_pre[i] * (1 + r[i])

    tc = transaction_rate * torch.norm(pi_new - pi_pre, p=1)

    # Update W_new with the transaction cost
    W_new = W_new - W_new * tc

    return W_new


class CustomLoss(nn.Module):
    def __init__(self, crra_coefficient):
        super(CustomLoss, self).__init__()
        self.crra_coefficient = crra_coefficient

    def forward(self, initial_wealth, initial_portfolio, upper_b, lower_b,returns,input_data):
        # Calculate the CRRA utility loss
        input_size = upper_b.shape[0]
        total_loss = 0
        #print(lower_b.shape,upper_b.shape)
        for _ in range(input_size):
            wealth = [initial_wealth]
            portfolio = [initial_portfolio]
            pi_star_arrays = []
            #print(portfolio)
            #constraint_loss = 0
            for i in range(T):
                w = wealth[-1]
                pi = portfolio[-1]
                pi_new = pi.clone().detach()
                pi_new = pi_new[:-1]

                closest_p_ind,__ = closest_probability_distn(input_data[_][i])
                pi_star = torch.tensor(optimal_pi_star[(closest_p_ind,i)],requires_grad=True)
                pi_star_arrays.append(pi_star)
                pi_star = pi_star[:-1]
                #print("Pi star: ",pi_star)
                #print(lb,ub)
                # print("Pi_star :",pi_star)
                u_b = upper_b[_][i]
                l_b = lower_b[_][i]

                #print("Boundaries: ",l_b,u_b)
                #constraint_loss+= torch.sum(torch.relu(l_b - u_b))*1000
                #print(constraint_loss)
                # print("Pi: ",pi_new)
                # print("Upper_delta: ",u_b)
                # print("Lower delta: ",l_b)
                # for j in range(n):
                #     pi_new[j] = max(max(-1.0,l_b[j]), min(min(u_b[j],1), pi_new[j]))
                u_b = torch.min(torch.ones(n),torch.add(pi_star,u_b))
                l_b = torch.max(-torch.ones(n),torch.add(pi_star,-l_b))
                pi_new = torch.max(l_b,torch.min(u_b,pi_new))
                # print("Upper boundary: ",u_b)
                # print("Lower_boundary: ",l_b)
                # print("Pi: ",pi_new)
                risk_free_allocation = torch.tensor([1-torch.sum(pi_new)],requires_grad=True)
                #print(risk_free_allocation)
                #print(risk_free_allocation)
                pi_new = torch.stack((pi_new,risk_free_allocation)).reshape(n+1)
                #print(pi_new)
                portfolio.append(torch.tensor(pi_new,requires_grad=True))
                #print(portfolio)
                new_wealth = newWealth_withTC(W=w,r = returns[_][i], pi_pre= pi,pi_new= pi_new)
                #print(new_wealth)
                wealth.append(new_wealth)

            #print(wealth)
            loss = -((wealth[-1]) ** self.crra_coefficient) / self.crra_coefficient 

            total_loss+= loss
            # print("Portfolio: ",portfolio)
            # print("Wealth: ",wealth)
            # print("Optimal_portfolios: ",pi_star_arrays)
        
        return total_loss/input_size

# Define your RNN cell with two neural networks
class CustomRNNCell(nn.Module):
    def __init__(self, input_size, output_size):
        super(CustomRNNCell, self).__init__()
        self.input_size = input_size
        #self.hidden_size = hidden_size
        self.output_size = output_size
        self.rnn = nn.RNN(input_size, hidden_size = output_size, num_layers = 1, batch_first=True)
        #self.fc1 = nn.Linear(input_size, output_size)
          #Network for "no-trade"
        # self.fc1 = nn.Sequential(
        #     nn.Linear(input_size,80,bias = True),
        #     nn.Sigmoid(),
        #     nn.Linear(80,40),
        #     nn.Sigmoid(),
        #     nn.Linear(40,20),
        #     nn.Sigmoid(),
        #     nn.Linear(20,output_size),
        #     nn.Sigmoid()
        # )
        # # # self.fc2 = nn.Linear(input_size, output_size)  # Network for "x-zone"
        # # self.fc2 = nn.Sequential(
        # #     nn.Linear(input_size,80),
        # #     nn.Tanh(),
        # #     nn.Linear(80,40),
        # #     nn.ReLU(),
        # #     nn.Linear(40,20),
        # #     nn.ReLU(),
        # #     nn.Linear(20,output_size),
        # #     nn.Tanh()
        # # )
        # self.fc2 = nn.Sequential(
        #     nn.Linear(input_size,80,bias = False),
        #     nn.Sigmoid(),
        #     nn.Linear(80,40,bias = False),
        #     nn.Sigmoid(),
        #     nn.Linear(40,20,bias = False),
        #     nn.Sigmoid(),
        #     nn.Linear(20,output_size,bias = False),
        #     nn.Sigmoid()
        # )

        # self.fc1 = nn.Sequential(
        #     nn.Linear(input_size, 80, bias=True),
        #     nn.Tanh(),  # ReLU activation for better sensitivity
        #     #nn.BatchNorm1d(80),  # Batch normalization
        #     nn.Linear(80, 40, bias=True),
        #     nn.Tanh(),  # ReLU activation
        #     #nn.BatchNorm1d(40),  # Batch normalization
        #     nn.Linear(40, 20, bias=True),
        #     nn.Tanh(),  # ReLU activation
        #     #nn.BatchNorm1d(20),  # Batch normalization
        #     nn.Linear(20, output_size),
        #     nn.Sigmoid()
        # )   

        # self.fc2 = nn.Sequential(
        #     nn.Linear(input_size, 80, bias=True),
        #     nn.Tanh(),  # Tanh activation
        #     #nn.BatchNorm1d(80),  # Batch normalization
        #     nn.Linear(80, 40, bias=True),
        #     nn.Tanh(),  # Tanh activation
        #     #nn.BatchNorm1d(40),  # Batch normalization
        #     nn.Linear(40, 20, bias=True),
        #     nn.Tanh(),  # Tanh activation
        #     #nn.BatchNorm1d(20),  # Batch normalization
        #     nn.Linear(20, output_size, bias=True),
        #     nn.Sigmoid()
        # )   

        #self.observed_boundaries = []

    def forward(self, x):
        input_to_nn= x.reshape(-1,N)
        #print(x.shape)
        # out_l = self.fc1(x*1000)
        # out_u = self.fc2(x*1000)
        # print(out_l)
        # print(out_u)
        #print(out.shape)
        # out_no_trade = self.fc1(out)
        # out_x_zone = self.fc2(out)
        #print(out_u.shape)
        # lower_b = out_l.reshape(-1,T,n)
        # upper_b = out_u.reshape(-1,T,n)
        #print(x.shape)
        # lower_b = torch.tanh(self.fc1(x))
        # upper_b = torch.tanh(self.fc2(x))
        #upper_b = torch.maximum(lower_b,upper_b)
        out,_ = self.rnn(x)
        out = torch.sigmoid(x)
        out_l = out[:,:,:n]
        out_u = out[:,:,n:]
        return torch.tensor(out_l,requires_grad=True),torch.tensor(out_u,requires_grad=True)

# Example usage of the custom loss function and RNN cell
input_size = N  # Replace with the appropriate input size
#hidden_size = 150  # Replace with the appropriate hidden size
output_size = 2*n # 2 output units, "no-trade" and "x-zone" decisions
crra_coefficient = 1  # CRRA coefficient, adjust as needed

rnn_cell = CustomRNNCell(input_size, output_size)
custom_loss = CustomLoss(crra_coefficient)

# Generate some example data
#input_data = torch.randn(T, input_size)  # (batch_size, sequence_length, input_size)
#target_data = torch.randn(T, output_size)  # (batch_size, sequence_length, output_size)
#regime_probs = torch.rand(T,2)  # Probability vector over regimes

n_samples = 1000
input_data = []
return_samples= []
cnt0 = 0
plot_x = []
for i in range(n_samples):
    sample = generate_monte_carlo_sample()
    if(sample[0][0]==0):
        cnt0+=1
    returns_for_this_sample = []
    for time in range(T):
        returns_for_this_sample.append(sample[time][1])
    return_samples.append(returns_for_this_sample)
    #return_samples.append()
    initial_p = np.random.rand(N)
    plot_x.append(initial_p[0])
    initial_p/=np.sum(initial_p)
    input_to_rnn = [initial_p]
    for time in range(T-1):
        p = input_to_rnn[-1]
        p_new = updateBelief_for_RNN(sample[time][1],p)
        #p_new = closest_probability_distn(p_new)[1]
        plot_x.append(p_new[0])
        input_to_rnn.append(p_new)
    input_data.append(input_to_rnn)

print(cnt0)
print(len(plot_x))
import matplotlib.pyplot as plt
plt.hist(plot_x,bins = 100)
plt.show()

print("Input data: ",input_data[0])
print("Return_Samples: ",return_samples[0])
input_data = torch.tensor(input_data,dtype= torch.float32,requires_grad=True)
#return_samples = torch.tensor(return_samples,dtype = torch.float32)

optimizer = optim.Adam(rnn_cell.parameters(), lr=0.0001)
#print(reverse_index)
# Forward pass and compute lossnp.ones(n+1)/(n+1)
for iter in range(10):
    lb,ub = rnn_cell(input_data)
    #print(lb.shape)
    #print(lb,ub)
    initial_wealth = torch.scalar_tensor(1000,requires_grad=True)
    initial_portfolio = torch.rand(n+1,requires_grad=True)
    initial_portfolio = initial_portfolio/torch.sum(initial_portfolio)
    loss = custom_loss(initial_wealth= initial_wealth,initial_portfolio = initial_portfolio,upper_b = ub,lower_b = lb,returns = np.array(return_samples),input_data = input_data)

    print("Loss: ",loss)
    # You can then use these losses to perform backpropagation and optimize your RNN cell using an optimizer like Adam.
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    


